In [ ]:
import sys
sys.path.append("..")

import random
import math
from io import BytesIO
from pathlib import Path
from collections import OrderedDict
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset, RandomSampler
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display

from src.datasets import *
from src.util.image import *
from src.util import *
from src.algo import *
from src.models.cnn import *

In [ ]:
SHAPE = (3, 64, 64)
dataset = TensorDataset(torch.load(f"../datasets/kali-uint8-{SHAPE[-2]}x{SHAPE[-1]}.pt"))
dataset = TransformDataset(dataset, dtype=torch.float, multiply=1./255.)
features = pd.read_pickle(f"../datasets/kali-uint8-{SHAPE[-2]}x{SHAPE[-1]}-features.df")

In [ ]:
px.line(features.iloc[:100])

In [ ]:
px.imshow(features.corr())

In [ ]:
for key in features.columns:
    df = features.sort_values(key, ascending=False)
    print(key)
    display(VF.to_pil_image(make_grid(
        [dataset[i][0] for i in df.index[:24]], nrow=24
    )))

In [ ]:
rows = []
for i in range(100):
    img = VF.to_pil_image(dataset[i][0])
    row = {}
    for q in range(0, 20, 1):
        row[f"q{q}"] = compressed_size(img, "jpeg", quality=q)
    rows.append(row)
df = pd.DataFrame(rows)
#px.line(df)
px.imshow(df.corr())

In [ ]:
rows = []
for i in range(100):
    img = VF.to_pil_image(dataset[i][0])
    row = {}
    for q, o in (
        (0, False), (1, False), (1, True), (4, False), (4, True), (9, False), (9, True),
    ):
        row[f"q{q}{o}"] = compressed_size(img, "png", compress_level=q, optimize=o)
    rows.append(row)
df = pd.DataFrame(rows)
df

In [ ]:
px.line(df)

In [ ]:
img=VF.to_pil_image(dataset[118][0])
img

In [ ]:
rows = []
ratioer = ImageCompressionRatio()
for i in range(1000, 2000):
    img = VF.to_pil_image(dataset[i][0])
    row = ratioer.all(img)
    row.update(ratioer.all(
        VF.gaussian_blur(img, kernel_size=[21, 21], sigma=10),
        suffix="-blur",
    ))
    rows.append(row)
df = pd.DataFrame(rows)
display(px.line(df))
px.imshow(df.corr())

In [ ]:
df2 = df.loc[df["png-low-blur"] < df["jpeg-low"]].copy()
df2.loc[:, "diff"] = df["jpeg-low"] - df["png-high"]
df2 = df2.sort_values("diff")
#df2
#px.line(df2)
VF.to_pil_image(make_grid([dataset[i][0] for i in df2.index]))